# 1. Initializations

## 1.1 General imports

In [ ]:
### data management
import pandas as pd
import numpy as np
# import statsmodels.api as sm

### graphical plotly basics
import plotly.graph_objects as go
import plotly.express as px
# for jupyter notebook display management
import plotly.io as pio
pio.renderers.default = "notebook"

## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

### 2.2.1 VELIB DISPO

In [ ]:
df_disp_velib_raw = dfc.load_dataset_from_config('velib_dispo_data', sep=';')

if df_disp_velib_raw is not None and isinstance(df_disp_velib_raw, pd.DataFrame):
    display(df_disp_velib_raw.head())
    dfc.log_general_info(df_disp_velib_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_disp_velib_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_disp_velib_raw))
    df_disp_velib = dfc.normalize_column_names(df_disp_velib_raw)
    display(df_disp_velib.head())

In [ ]:
df_cpt_velib_desc = df_disp_velib.select_dtypes(include=np.number).describe()
display(df_cpt_velib_desc)
df_cpt_velib_cr = df_disp_velib.select_dtypes(include=np.number).corr()
display(df_cpt_velib_cr)

### 2.2.2 VELIB COMPTAGE

In [ ]:
df_cpt_velib_raw = dfc.load_dataset_from_config('velib_comptage_data', sep=';')

if df_cpt_velib_raw is not None and isinstance(df_cpt_velib_raw, pd.DataFrame):
    display(df_cpt_velib_raw.head())
    dfc.log_general_info(df_cpt_velib_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_cpt_velib_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_cpt_velib_raw))
    df_cpt_velib = dfc.normalize_column_names(df_cpt_velib_raw)
    display(df_cpt_velib.head())

In [ ]:
df_cpt_velib_desc = df_disp_velib.select_dtypes(include=np.number).describe()
display(df_cpt_velib_desc)
df_cpt_velib_cr = df_disp_velib.select_dtypes(include=np.number).corr()
display(df_cpt_velib_cr)

## 2.2 Data quality refinement

In [ ]:
# original backup and duplicates management
df_disp_velib_orig = df_disp_velib.copy()
df_disp_velib = df_disp_velib.drop_duplicates()
df_cpt_velib_orig = df_cpt_velib.copy()
df_cpt_velib = df_cpt_velib.drop_duplicates()

## 2.3 Data combination and rework

# 2. Data Viz' and Analysis

## 2.1 General Data Viz'

## 2.2 Quantitative mono variable distribution

## 2.3 Qualitative mono variable distribution

## 2.4 Qualitative multi variable distribution

## 2.5 Quantitative multi variable correlation